<a href="https://colab.research.google.com/github/sattviksahai/CS583B_Recitation/blob/master/Recitation9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Read data ##

In [6]:
with open('corpus.txt') as f:
  content = f.read().replace('\n', " ").split(".")

## Remove Punctuations ##

In [10]:
import re

clean_lines=[]
for line in content:
  clean_lines.append(re.sub("[\.\;\’\“\”\\n\—\,\‐\?\!\:\‘\(\)\_\--]", " ", line))

## Tokenize ##

In [16]:
train_tokens=[]
for line in clean_lines:
  tokens = "<s> {} <\s>".format(line).split(" ")
  train_tokens.append([token for token in tokens if token != ''])

## Create Vocabulary ##

In [19]:
vocabulary = {}
ind = 0
for line in train_tokens:
  for word in line:
    if word not in vocabulary:
      vocabulary[word] = ind
      ind += 1

In [22]:
len(vocabulary)

18447

## Get bigram counts ##

In [26]:
import numpy as np
from tqdm.notebook import tqdm

bigrams = np.zeros((len(vocabulary), len(vocabulary)))
unigrams = np.zeros((1, len(vocabulary)))
for line in tqdm(train_tokens):
  for i in range(len(line)-1):
    prev_word = vocabulary[line[i]]
    current_word = vocabulary[line[i+1]]
    bigrams[prev_word, current_word] += 1
    unigrams[0, prev_word] += 1
  unigrams[0, current_word] += 1
bigrams = bigrams/unigrams

In [27]:
bigrams

array([[0.        , 1.        , 0.78823529, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.01176471, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

## Generate Sentences ##

Step 1: Randomly sample a starting word from vocabulary

Step 2: Sample next word based on the distribution of $P(w_i|w_{i-1})$

Step 3: Continue sampling until a stop token is predicted.

In [41]:
def generate_sentence(first_word):
  keys = list(vocabulary.keys())
  query = first_word
  sentence=[]
  sentence.append(first_word)
  for i in range(50):
    distribution = bigrams[vocabulary[query],:]
    # print(distribution)
    predicted_word = keys[np.random.choice(np.arange(0, len(distribution)), p=distribution/np.sum(distribution))]
    # print(predicted_word)
    query = predicted_word
    sentence.append(predicted_word)
    if predicted_word == "<\s>":
      break
  return " ".join(sentence)

In [45]:
import random

first_word = random.choice(list(vocabulary.keys()))
# print(first_word)
generate_sentence(first_word)

'fairy land for" Mrs Clay\'s freckles were incessantly challenging his loins only warded off pleasantly handled in protecting her start forth bitter philippic "Miss Price plaintively "it confesses no harsher name \'Lydia Wickham "his observations seemingly unhandsome <\\s>'